<h1 style="font-size: 22px; line-height: 100%; text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142); border-radius: 10px;">
Sub-network Exploration with BootstrapNAS
</h1>

This notebook demonstrates how to load the Shears super-network and extract particular subnetworks so that users can apply their own search solutions to find high-performing subnetworks. We will use an existing super-network from our released models - [shears-llama-7b-50-math-super](https://huggingface.co/IntelLabs/shears-llama-7b-50-math-super).

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Imports and Settings
</h3>

In [ ]:
import os
import torch
from peft import PeftModel
from search.supernet import ShearsSuperNet
from transformers import AutoModelForCausalLM

from nncf import NNCFConfig
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.elasticity_dim import ElasticityDim
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.multi_elasticity_handler import SubnetConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

SHEARS_PATH = "./shears-llama-7b-50-math-super"
BASE_MODEL_PATH = f"{SHEARS_PATH}/base_model"
ADAPTER_MODEL_PATH = f"{SHEARS_PATH}/adapter_model"
NNCF_CONFIG = "../nncf_config/unified_math/nncf_shears_llama_7b_sparsity50.json"


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Loading an existing super-network
</h3>

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH,trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL_PATH)
nncf_config = NNCFConfig.from_json(NNCF_CONFIG)
supernet = ShearsSuperNet.from_checkpoint(model, nncf_config, supernet_elasticity_path=None, supernet_weights_path=None)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Explore the Search Space
</h3>



In [ ]:
supernet.get_search_space()

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Sample sub-networks
</h3>

In [ ]:
# maximal subnetwork
supernet.activate_maximal_subnet()
supernet.get_active_config()
supernet.get_macs_for_active_config()

In [ ]:
# heuristic subnetwork
supernet.activate_heuristic_subnet()
supernet.get_active_config()
supernet.get_macs_for_active_config()

In [ ]:
# minimal subnetwork
supernet.activate_minimal_subnet()
supernet.get_active_config()
supernet.get_macs_for_active_config()

In [ ]:
# random subnetwork
import random
subnet_config = SubnetConfig()
search_space = supernet.get_search_space()
subnet_config[ElasticityDim.WIDTH] = {i: random.choice(space) for i, space in search_space[ElasticityDim.WIDTH.value].items()}

supernet.activate_config(subnet_config)
supernet.get_active_config()
supernet.get_macs_for_active_config()

# extract and save the active sub-adapter
supernet.extract_and_save_active_sub_adapter(super_adapter_dir=ADAPTER_MODEL_PATH, sub_adapter_dir=os.path.join(ADAPTER_MODEL_PATH, "sub_adapter"))
